In [26]:
import requests
import pandas as pd
import numpy as np
import json
import time

In [2]:
apiKey = "IaF6zOUaj8VG4ambmUwOodEFaxBnrR3A"
apiUrl = "https://aeroapi.flightaware.com/aeroapi/"
auth_header = {'x-apikey':apiKey}

# GET flights from DMK airport

In [3]:
Airport_id = 'DMK'
Airport_start = '2022-12-01T00:00:00Z'
Airport_end = '2022-12-04T00:00:00Z'

# Y-M-D
# 1) 2022-12-01 - 2022-12-04


In [4]:
#----- get flight data from DMK airport -----#

# response = requests.get(apiUrl + f"airports/{Airport_id}/flights?start=2022-12-01T00%3A00%3A00Z&end=2022-12-04T00%3A00%3A00Z",
#  headers=auth_header)

# if response.status_code == 200:
#     print(response.json())
# else:
#     print("Error executing request")

In [5]:
print(json.dumps(response.json(), indent=4))

NameError: name 'response' is not defined

In [ ]:
Arrivals_FlightID = []
Departures_FlightID = []

In [ ]:
#----- add flight data from DMK airport -----#

for i in response.json()['arrivals']:
    Arrivals_FlightID.append(i['fa_flight_id'])
    
for i in response.json()['departures']:
    Departures_FlightID.append(i['fa_flight_id'])

In [ ]:
for item in Departures_FlightID:
    print(item)

AIQ642-1669939380-schedule-0476
NOK340-1669938420-schedule-0407
AIQ3437-1669938420-schedule-0400
AIQ311-1669938420-schedule-0399
AIQ3213-1669938420-schedule-0403
NOK122-1669938420-schedule-0392
NOK900-1669937520-schedule-0590
AIQ3102-1669937520-schedule-0604
AIQ3188-1669938420-schedule-0397
AIQ3029-1669937520-schedule-0599
TLM702-1669937520-schedule-0574
TLM506-1669936380-schedule-0442
NOK100-1669936380-schedule-0453
AIQ3252-1669936380-schedule-0445
AIQ396-1669936380-schedule-0404


# GET data from flight ID

In [ ]:
#----- get data from flight ID -----#
Dict_flights = {}

i = 1
for id in Departures_FlightID:
    # response = requests.get(apiUrl + f"flights/{id}/track",
    # headers=auth_header)

    if response.status_code == 200:
        print(i)
        #----- Add data to dict -----#
        Dict_flights[id] = pd.DataFrame.from_dict(response.json()['positions'])
        
        #----- interpolate 1 sec -----#
        Dict_flights[id]['timestamp'] = pd.to_datetime(Dict_flights[id].timestamp)
        nidx = np.arange(Dict_flights[id]['timestamp'][0], Dict_flights[id]['timestamp'].iloc[-1], 1000000 )
        nidx = pd.to_datetime(nidx)

        Dict_flights[id]['timestamp'] = Dict_flights[id]['timestamp'].round('S').dt.tz_localize(None)
        Dict_flights[id].set_index('timestamp', inplace=True)
        Dict_flights[id] = Dict_flights[id].reindex(Dict_flights[id].index.union(nidx))

        #----- interpolate ['altitude', 'groundspeed', 'heading', 'latitude', 'longitude'] -----#
        Dict_flights[id] = Dict_flights[id][['fa_flight_id', 'altitude', 'groundspeed', 'heading', 'latitude', 'longitude']]
        Dict_flights[id]['altitude'] = pd.to_numeric(Dict_flights[id]['altitude'])
        Dict_flights[id]['groundspeed'] = pd.to_numeric(Dict_flights[id]['groundspeed'])
        Dict_flights[id]['heading'] = pd.to_numeric(Dict_flights[id]['heading'])
        Dict_flights[id] = Dict_flights[id].interpolate(method='time',limit_direction='both',limit=100)
        Dict_flights[id]['fa_flight_id'] = id

        # df = pd.DataFrame.from_dict(response.json()['positions'])
        # df['fa_flight_id'] = id
        # df.to_csv(f'flights_data/{id}.csv',index = False)
            
    else:
        print("Error executing request")
        
    i+=1
    time.sleep(7)

1
2
Error executing request
Error executing request
Error executing request
6
7
8
9
10
11
12
13
14
15


In [ ]:
print(json.dumps(response.json(), indent=4))

{
    "positions": [
        {
            "fa_flight_id": null,
            "altitude": 4,
            "altitude_change": "C",
            "groundspeed": 179,
            "heading": 208,
            "latitude": 13.89944,
            "longitude": 100.59736,
            "timestamp": "2022-12-03T23:57:52Z",
            "update_type": "A"
        },
        {
            "fa_flight_id": null,
            "altitude": 10,
            "altitude_change": "C",
            "groundspeed": 184,
            "heading": 208,
            "latitude": 13.88798,
            "longitude": 100.59111,
            "timestamp": "2022-12-03T23:58:08Z",
            "update_type": "A"
        },
        {
            "fa_flight_id": null,
            "altitude": 17,
            "altitude_change": "C",
            "groundspeed": 186,
            "heading": 208,
            "latitude": 13.87573,
            "longitude": 100.58441,
            "timestamp": "2022-12-03T23:58:24Z",
            "update_type": "A"
    

In [6]:
df = pd.read_csv(f'flights_data/{id}.csv')
df.head()

OSError: [Errno 22] Invalid argument: 'flights_data/<built-in function id>.csv'

# test ไม่ใช้จ้า

In [99]:
# Dict_flights = {}

# Dict_flights['AIQ642-1669939380-schedule-0476'] = pd.read_csv(f'flights_data/AIQ642-1669939380-schedule-0476.csv')
# Dict_flights['AIQ642-1669939380-schedule-0476']['fa_flight_id'] = 1

# Dict_flights['NOK340-1669938420-schedule-0407'] = pd.read_csv(f'flights_data/NOK340-1669938420-schedule-0407.csv')
# Dict_flights['NOK340-1669938420-schedule-0407']['fa_flight_id'] = 2

# Dict_flights['AIQ642-1669939380-schedule-0476'].head()

,fa_flight_id,altitude,altitude_change,groundspeed,heading,latitude,longitude,timestamp,update_type
0,1,4,C,179,208,13.89944,100.59736,2022-12-03T23:57:52Z,A
1,1,10,C,184,208,13.88798,100.59111,2022-12-03T23:58:08Z,A
2,1,17,C,186,208,13.87573,100.58441,2022-12-03T23:58:24Z,A
3,1,23,C,190,233,13.86502,100.57491,2022-12-03T23:58:40Z,A
4,1,28,C,196,276,13.86214,100.56075,2022-12-03T23:58:56Z,A


In [96]:
Dict_flights['AIQ642-1669939380-schedule-0476']

Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'] = pd.to_datetime(Dict_flights['AIQ642-1669939380-schedule-0476'].timestamp)

nidx = np.arange(Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'][0], Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'].iloc[-1], 1000000 )
nidx = pd.to_datetime(nidx)
# nidx

Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'] = Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'].round('S').dt.tz_localize(None)
Dict_flights['AIQ642-1669939380-schedule-0476'].set_index('timestamp', inplace=True)
Dict_flights['AIQ642-1669939380-schedule-0476'] = Dict_flights['AIQ642-1669939380-schedule-0476'].reindex(Dict_flights['AIQ642-1669939380-schedule-0476'].index.union(nidx))

Dict_flights['AIQ642-1669939380-schedule-0476'] = Dict_flights['AIQ642-1669939380-schedule-0476'][['altitude', 'groundspeed', 'heading', 'latitude', 'longitude']]
Dict_flights['AIQ642-1669939380-schedule-0476']['altitude'] = pd.to_numeric(Dict_flights['AIQ642-1669939380-schedule-0476']['altitude'])
Dict_flights['AIQ642-1669939380-schedule-0476']['groundspeed'] = pd.to_numeric(Dict_flights['AIQ642-1669939380-schedule-0476']['groundspeed'])
Dict_flights['AIQ642-1669939380-schedule-0476']['heading'] = pd.to_numeric(Dict_flights['AIQ642-1669939380-schedule-0476']['heading'])
Dict_flights['AIQ642-1669939380-schedule-0476'] = Dict_flights['AIQ642-1669939380-schedule-0476'].interpolate(method='time',limit_direction='both',limit=50)

Dict_flights['AIQ642-1669939380-schedule-0476']

C:\Users\Nopnpr\AppData\Local\Temp\ipykernel_20784\1800424518.py:5: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  nidx = np.arange(Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'][0], Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'].iloc[-1], 1000000 )


,altitude,groundspeed,heading,latitude,longitude
2022-12-03 23:57:52,4.000,179.0000,208.0,13.899440,100.597360
2022-12-03 23:57:53,4.375,179.3125,208.0,13.898724,100.596969
2022-12-03 23:57:54,4.750,179.6250,208.0,13.898008,100.596579
2022-12-03 23:57:55,5.125,179.9375,208.0,13.897291,100.596188
2022-12-03 23:57:56,5.500,180.2500,208.0,13.896575,100.595798
...,...,...,...,...,...
2022-12-04 01:20:56,11.000,147.0000,107.0,21.243370,105.729980
2022-12-04 01:20:57,11.250,146.0000,107.0,21.243182,105.730627
2022-12-04 01:20:58,11.500,145.0000,107.0,21.242995,105.731275
2022-12-04 01:20:59,11.750,144.0000,107.0,21.242807,105.731922
